In [15]:
# Packages
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from scipy.stats import norm
import yfinance as yf

# Dataset
# Set start and end date
start_date = '2022-04-01'
end_date = '2024-03-31'

# Code of the Index or Stocks
code = ['CL=F', 'ES=F', 'GC=F', 'NG=F', 'NQ=F', '^GSPC', '^IXIC', 'YM=F', 'RTY=']

# Download data
df = yf.download(code, start_date, end_date)
df = df['Adj Close']

# Daily return daily prices and remove any NA
ret = np.log(df[code]/df[code].shift(1))
ret = ret.dropna()


[*********************100%%**********************]  5 of 5 completed


In [17]:
# Calculate the log return mean and covariance of each stock
ret.mean()*252

Ticker
CL=F   -0.089186
ES=F    0.078900
GC=F    0.072817
NG=F   -0.593184
NQ=F    0.109617
dtype: float64

In [19]:
# Variance covariance matrix
ret.cov()*252

Ticker,CL=F,ES=F,GC=F,NG=F,NQ=F
Ticker,,,,,
CL=F,0.141219,0.011016,0.008659,0.037537,0.008203
ES=F,0.011016,0.033916,0.004194,0.019037,0.043292
GC=F,0.008659,0.004194,0.018571,0.003133,0.004356
NG=F,0.037537,0.019037,0.003133,0.628427,0.017220
NQ=F,0.008203,0.043292,0.004356,0.017220,0.060501


In [25]:
# Single run for a random allocation
np.random.seed(101)

# Stock Columns
print('Stocks')
print(df.columns)
print('\n')

# Create Random Weights
print('Creating Random Weights')
weights = np.array(np.random.random(9))
print(weights)
print('\n')

# Rebalance Weights
print('Rebalance to sum to 1.0')
weights = weights / np.sum(weights)
print(weights)
print('\n')

# Expected Return
print('Expected Portfolio Return')
exp_ret = np.sum(ret.mean() * weights) *252
print(exp_ret)
print('\n')

# Expected Variance
print('Expected Volatility')
exp_vol = np.sqrt(np.dot(weights.T, np.dot(ret.cov() * 252, weights)))
print(exp_vol)
print('\n')

# Sharpe Ratio
SR = exp_ret/exp_vol
print('Sharpe Ratio')
print(SR)

Stocks
Index(['CL=F', 'ES=F', 'GC=F', 'NG=F', 'NQ=F'], dtype='object', name='Ticker')


Creating Random Weights
[0.51639863 0.57066759 0.02847423 0.17152166 0.68527698 0.83389686
 0.30696622 0.89361308 0.72154386]


Rebalance to sum to 1.0
[0.10921307 0.12069041 0.00602201 0.03627509 0.14492913 0.17636073
 0.06492024 0.1889901  0.15259921]


Expected Portfolio Return


ValueError: operands could not be broadcast together with shapes (5,) (9,) 

In [23]:
# Multiple run for a random allocation
num_ports = 100000

all_weights = np.zeros((num_ports,9))
ret_arr = np.zeros(num_ports)
vol_arr = np.zeros(num_ports)
sharpe_arr = np.zeros(num_ports)

for ind in range(num_ports):
    # Create Random Weights
    weights = np.array(np.random.random(9))

    # Rebalance Weights
    weights = weights / np.sum(weights)
    
    # Save Weights
    all_weights[ind,:] = weights

    # Expected Return
    ret_arr[ind] = np.sum((ret.mean() * weights) *252)

    # Expected Variance
    vol_arr[ind] = np.sqrt(np.dot(weights.T, np.dot(ret.cov() * 252, weights)))

    # Sharpe Ratio
    sharpe_arr[ind] = ret_arr[ind]/vol_arr[ind]

NameError: name 'logreturn' is not defined

In [ ]:
# Get the maximum value of sharpe ratio and the location
SR_max = sharpe_arr.max()
SR_max_loc = sharpe_arr.argmax()
print(SR_max)
print(SR_max_loc)
print(weights)

In [ ]:
# The weights corresponding to that location
opt_weights = all_weights[SR_max_loc,:]
print(opt_weights)

In [ ]:
# Plot all the portfolio combination runs on a graph and point out the maximum sharpe ratio.
max_sr_ret = ret_arr[SR_max_loc]
max_sr_vol = vol_arr[SR_max_loc]
plt.figure(figsize=(17,9))
plt.scatter(vol_arr,ret_arr,c=sharpe_arr,cmap='plasma')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility')
plt.ylabel('Return')
# Add red dot for max SR
plt.scatter(max_sr_vol,max_sr_ret,c='red',s=50,edgecolors='black')